First notebook. Any suggestions on how to improve would be appreciated

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Data = pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")
Data.columns

In [ ]:
Data.head()

In [ ]:
X = Data.drop(['id','Unnamed: 32', 'diagnosis'],axis=1)
X

In [ ]:
Data["diagnosis"].value_counts()

In [ ]:
Y = Data.diagnosis
Y= Y.apply(lambda x: 1 if x == "M" else 0)
Y

Normalizing the data


In [ ]:
X=((X-X.min())/(X.max()-X.min()))
X

Preparing model:
Since the label (diagnosis) is discret, RandomForestClassifier will be used

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:

x_train,x_test, y_train, y_test = train_test_split(X,Y,test_size = .2,random_state = 0)
print(x_train.shape,y_train.shape, x_test.shape,y_test.shape)


In [ ]:
model = RandomForestClassifier(n_estimators = 250,random_state=1)
model.fit(x_train,y_train)
preds = model.predict(x_test)
acc_test = metrics.accuracy_score(y_test, preds)

print ('The accuracy on test data is %s' % (round(acc_test,2)))


Attempting to determine which feature is the most important

In [ ]:
import matplotlib.pyplot as plt
import time

In [ ]:
start_time = time.time()
std = np.std([
    tree.feature_importances_ for tree in model.estimators_], axis=0)
elapsed_time = time.time() - start_time

print(f"Elapsed time to compute the importances: "
      f"{elapsed_time:.3f} seconds")
feature_imp = pd.Series(model.feature_importances_,index=X.columns).sort_values(ascending=False)

In [ ]:
fig, ax = plt.subplots()
feature_imp.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()


In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
print(x_train.shape)

In [ ]:
nn_model = Sequential()
nn_model.add(Input(shape= (30,)))
nn_model.add(Dense(60, activation='relu'))
nn_model.add(Dense(60, activation='relu'))
nn_model.add(Dense(110, activation='relu'))
nn_model.add(Dense(400, activation='relu'))
nn_model.add(Dense(150, activation='relu'))
nn_model.add(Dense(1,  activation = 'sigmoid'))
nn_model.summary()

In [ ]:
nn_model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = nn_model.fit(x_train,y_train, epochs = 65,verbose = 1,validation_data = (x_test,y_test))

In [ ]:
preds = nn_model.predict(x_test)
preds = np.around(preds)
acc_test = metrics.accuracy_score(y_test, preds)

print ('The accuracy on test data is %s' % (round(acc_test,2)))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')